In [4]:
# import packages
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from config import username, password
import datetime
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict

In [5]:
# init browser
executable_path = {'executable_path': '/Users/yangsun/Downloads/chromedriver 5'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
# visit url
login_url='https://www.glassdoor.com/profile/login_input.htm'
browser.visit(login_url)

In [7]:
# sign in
browser.find_by_id('inlineUserEmail').fill(username)
browser.find_by_id('inlineUserPassword').fill(password)
browser.find_by_xpath('//button[@type="submit"]').click()

In [8]:
# go to new url
url='https://www.glassdoor.com/Salary/Panda-Express-Salaries-E5659.htm'
browser.visit(url)

In [9]:
def select_area(area_name):
    """select area from selector
    """
    browser.find_by_xpath('//button[@class="gd-ui-button d-none d-lg-inline-block css-pitbid"]').click()
    browser.find_by_xpath('//span[@class="SVGInline arrowDown"]')[1].click()
    browser.find_by_text(area_name).first.click()

In [29]:
area_name='- Los Angeles, CA Area'
select_area(area_name)
time.sleep(2)

In [11]:
html=browser.html
soup = bs(html, 'html.parser')
# list of locations from 23rd starting
soup.find_all('li',class_='dropdownOption')[23]

<li aria-selected="false" class="dropdownOption" id="option_N,1" role="option"><div class="checkmark"><span alt="" class="SVGInline"><svg class="SVGInline-svg" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg"><path d="M9.89 15.76l-2.64-2.363a.793.793 0 010-1.157.884.884 0 011.211 0l2.039 1.785 5.039-5.785a.884.884 0 011.21 0 .793.793 0 010 1.157L11.1 15.76a.884.884 0 01-1.21 0z" fill="currentColor" fill-rule="evenodd"></path></svg></span></div><span class="dropdownOptionLabel css-1b6bxoo">United States - All Cities</span></li>

In [82]:
# total page number
page=int(soup.find_all('div',class_='paginationFooter')[0].text.split(' ')[-2])

In [12]:
# all pay list
pay_lists=soup.find_all('div',class_='css-1u4lhyp')

In [61]:
# only look for 
title_list = ['general manager',
              'assistant manager',
              'counter help',
              'counter helper',
              'shift leader',
              'cook',
              'shift lead',
              'kitchen help',
              'kitchen helper',
              'lead counter',
              'lead counter help',
              'front of house',
              'back of house',
              'cashier',
              'crew member',
              'chef',
              'pic',
              'FOH team member',
              'kitchen staff',
              'FOH']

In [72]:
def get_frontend_info(pay_list):
    """Extract front end job title information
           - front end title
           - avg total pay
           - additional pay
       Return: 
           {front end title:[avg base pay, base pay, additional pay]}
    """
    # front_end_info= {front end title:[avg base pay, base pay, additional pay, Number of reported salaries]}
    front_end_info=defaultdict(list)
    title=pay_list.a.strong.text
    # pay
    pay=pay_list.span.strong.text
    # additional pay
    tag=pay_list.find_all('span',text='Additional Pay')
    # check if additional pay tag exist
    if len(tag)==0:
        additional_pay=0
    else:
        additional_pay=tag[0].find_parent('div').strong.text
    # reports
    reports=pay_list.find_all('span',class_='css-kkqchx')[-1].text
    front_end_info[title]=[pay,additional_pay,reports]
    return front_end_info

In [75]:
def next_page():
    """click next page button
    """
    browser.find_by_xpath('//button[@class="nextButton css-1hq9k8 e13qs2071"]').click()

In [87]:
def get_current_page_info(pay_lists,res:list):
    """Extract current page pay information
    """
    for pay_list in pay_lists:
        title=pay_list.a.strong.text
        if title.lower() not in title_list:
            continue
        # get front data
        front_end_info=get_frontend_info(pay_list)
        res.append(front_end_info)
    return res

In [89]:
# get the first 20 pages data
final_res=[]
for p in range(20):
    get_current_page_info(pay_lists,final_res)

In [116]:
# create a list for panda dataframe
t=[list(d.keys())+list(d.values())[0] for d in final_res]

In [118]:
import pandas as pd
final_data=pd.DataFrame(t,columns=['title','Average total pay','additional pay','creditbility'])
final_data.to_csv('LA,CA Panda pay.csv')

In [18]:
# go to each title
sub_url=url+pay_lists[0].a['href']
browser.visit(sub_url)
sub_html=browser.html

In [28]:
title=pay_lists[0].a.strong.text
title

'Counter Help'

In [ ]:
# get inside pay rate information
def get_detailed_pay(sub_html):
    """Extract detailed pay rate after clicking each title in the frontend
       Return: (dict)
           - detailed pay
           - additional pay
           - Related company pay
    """
    results=defaultdict(list)
    

In [219]:
# go to sub_url
browser.visit(sub_url)
sub_html=browser.html

In [22]:
# based pay average and additional pay
soup_sub=bs(sub_html,'html.parser')
avg_base_pay=soup_sub.find_all('div',class_='col-12 col-lg-4')[0].h2.text
additional_pay=soup_sub.find_all('div',class_='col-12 col-lg-7 d-flex align-items-baseline')[0].h2.text


In [24]:
# compare with market
market_comparison=soup_sub.find_all('strong',class_='css-1uyte9r css-1qxtz39 css-fkag2m css-ztwr07 eb8fmw1')[0].text

In [136]:
# related salaries with others
text='Related Salaries'
tag=soup_sub.find("h3", text=text)
container=tag.find_parent('div')

In [148]:
# container information
s_c=container.find_all('div',class_='row no-gutters mx-0 py align-items-center css-1u4lhyp')
title=s_c[0].p.a.text
name=s_c[0].find_all('p',class_='m-0 strong')[0].text
pay=s_c[0].find_all('strong',class_='d-block css-1djk842')[0].text

In [151]:
# get all container information
# company name, title, pay
# related company={company name:[title, pay]}
related_company=defaultdict(list)
for sub in s_c:
    # pay title
    title=sub.p.a.text
    # company name
    name=sub.find_all('p',class_='m-0 strong')[0].text
    # pay
    pay=sub.find_all('strong',class_='d-block css-1djk842')[0].text
    # append to list
    related_company[name]=[title, pay]
related_company

defaultdict(list,
            {"Jersey Mike's": ['Counter Help - Hourly', '$16 - $17'],
             'Hanmi Financial': ['Counter Help', '$44K - $47K'],
             'Walters Wholesale Electric': ['Counter Help - Hourly',
              '$20 - $22']})

In [154]:
# go back to previous url
browser.visit(url)